# Setting up a single-node Kubernetes cluster with `kind`
In this notebook, we will set up a single-node Kubernetes cluster on the machine running the Jupyter kernel and test it. The only prerequisite is that Docker is running. All other required components will be downloaded and installed in `$PWD/download`.

**TODO:** Write a script that automates these steps, which can then be used for other k8s-related notebooks

## Create required directories and update `PATH`

In [1]:
mkdir -p download
export DOWNLOAD_DIR=$PWD/download

if [[ ":$PATH:" != *":$DOWNLOAD_DIR:"* ]]; then
    export PATH=$PWD/download:$PATH
fi

## Download *kubectl*
For any interaction with a Kubernetes cluster, we need *kubectl*.

In [2]:
if [ ! -f "download/kubectl" ]; then
    echo "Downloading kubectl..."
    curl -sLo download/kubectl https://storage.googleapis.com/kubernetes-release/release/$(curl -s https://storage.googleapis.com/kubernetes-release/release/stable.txt)/bin/linux/amd64/kubectl
    chmod u+x download/kubectl
fi

## Download *kind*
*kind* (Kubernetes in Docker) is a tool that can be used to set up a local Kubernetes cluster for testing purposes easily: https://github.com/kubernetes-sigs/kind

In [3]:
if [ ! -f "download/kind" ]; then
    echo "Downloading kind..."
    curl -sLo download/kind https://github.com/kubernetes-sigs/kind/releases/download/v0.5.1/kind-$(uname)-amd64
    chmod +x download/kind
fi

## Run `kind` without arguments to see the available commands

In [4]:
kind

kind creates and manages local Kubernetes clusters using Docker container 'nodes'

Usage:
  kind [command]

Available Commands:
  build       Build one of [base-image, node-image]
  completion  Output shell completion code for the specified shell (bash or zsh)
  create      Creates one of [cluster]
  delete      Deletes one of [cluster]
  export      exports one of [logs]
  get         Gets one of [clusters, nodes, kubeconfig, kubeconfig-path]
  help        Help about any command
  load        Loads images into nodes
  version     prints the kind CLI version

Flags:
  -h, --help              help for kind
      --loglevel string   logrus log level [panic, fatal, error, warning, info, debug, trace] (default "warning")
      --version           version for kind

Use "kind [command] --help" for more information about a command.


## Create a Kubernetes cluster with the default name *kind*

In [5]:
kind create cluster

Creating cluster "kind" ...
 ✓ Ensuring node image (kindest/node:v1.15.3) 🖼
 ✓ Preparing nodes 📦 
 ✓ Creating kubeadm config 📜 
 ✓ Starting control-plane 🕹️ 
 ✓ Installing CNI 🔌 
 ✓ Installing StorageClass 💾 
Cluster creation complete. You can now use the cluster with:

export KUBECONFIG="$(kind get kubeconfig-path --name="kind")"
kubectl cluster-info


## Set up `KUBECONFIG` environment variable and get cluster information 

In [6]:
export KUBECONFIG="$(kind get kubeconfig-path --name="kind")"
kubectl cluster-info

Kubernetes master is running at https://127.0.0.1:38699
KubeDNS is running at https://127.0.0.1:38699/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


## Get the Kubernetes resources in the cluster

In [7]:
kubectl get all --all-namespaces



NAMESPACE     NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)                  AGE
default       service/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP                  7s
kube-system   service/kube-dns     ClusterIP   10.96.0.10   <none>        53/UDP,53/TCP,9153/TCP   5s

NAMESPACE     NAME                        DESIRED   CURRENT   READY   UP-TO-DATE   AVAILABLE   NODE SELECTOR                 AGE
kube-system   daemonset.apps/kindnet      0         0         0       0            0           <none>                        3s
kube-system   daemonset.apps/kube-proxy   0         0         0       0            0           beta.kubernetes.io/os=linux   5s

NAMESPACE     NAME                      READY   UP-TO-DATE   AVAILABLE   AGE
kube-system   deployment.apps/coredns   0/2     0            0           5s







## Note that the Kubernetes cluster is running in a single Docker container

In [8]:
docker ps

CONTAINER ID        IMAGE                  COMMAND                  CREATED             STATUS              PORTS                                  NAMES
e70610949581        kindest/node:v1.15.3   "/usr/local/bin/entr…"   32 seconds ago      Up 29 seconds       38699/tcp, 127.0.0.1:38699->6443/tcp   kind-control-plane


## Finally, delete the created cluster and delete the (now invalid) contents of `KUBECONFIG`


In [9]:
unset KUBECONFIG
kind delete cluster

Deleting cluster "kind" ...
